<div dir="rtl">
<font face="B Zar" size=18>

تغییرات فایل تست:
<ul>
<li>
برای طول posting_list از تابع length_of_posting_list به جای len استفاده کردم که آیتم‌های خالی رو حساب نمی‌کنه
</li>
<li>
در آدرس‌ها data/ را از اول data/Persian.xml برداشتم
</li>
<li>
به آخر storage/backup یک .txt اضافه کردم
</li>
<li>
در functions[f](doc) به جای doc، str(doc) گذاشتم
</li>
</ul>

تغییرات کد:
<ul>
<li>
در تابع detailed_search و هر کدام از ۴ تابع evaluation، یک خط پرینت نتیجه بود که کامنت کردم
</li>
<li>
تابع‌های search و detailed_search بیست تا نتیجه‌ی اول را برمی‌گرداندند که ۲۰ را به ۱۵ تغییر دادم
</li>
<li>
در add_document_to_indexes و delete_document_from_indexes آیدی داک به آخر مسیر اضافه شده (خط ۸۹ و ۱۱۲) آن را پاک کردم و به جای آن بعد از خط ۹۶ و ۱۱۹ یک if pid == doc_num گذاشتم تا داک را پیدا کند. تا آخر تابع جزو if می‌شود
</li>
<li>
در خط ۳۷۶ و ۴۲۴، مخرج را با $2^{-16}$ جمع کردم. (تابع آماده‌ی precision_recall_fscore_support هم وقتی precision و recall با هم 0 می‌شوند به صورت پیش‌فرض هشدار می‌دهد و عددی را بر‌نمی‌گرداند)
</li>
</ul>
</font>
</div>


In [ ]:
def length_of_posting_list(p):
    return len([a1 for a1, v in p.items() if len(v["title"]) > 0 or len(v["text"]) > 0])


word1 = 'فکری'
doc_id = 3014

word2 = 'هیلاندراس'
doc_id2 = 6752
bigram = 'لا'


def get_count(l):
    i = [1 for _, t in l.items() for q in t['text']]
    j = [1 for _, t in l.items() if 'title' in t.keys() for q in t['title']]
    return len(i) + len(j)


def test_prepare_text():
    print("\n============ testing 'prepare_text' =============================================")
    raw_text = "کتابهای مناسبی نوشته شوند ! در راستای ارتقای . سطح آموزش کشور ؟ تلاش‌های زیادی صورت می‌گیرد"
    prepared_text = prepare_text(raw_text)

    print("prepared text is :", prepared_text, "with length:", len(prepared_text))


test_prepare_text()


def test_get_posting_list():
    print("\n============ testing 'get_posting_list' =========================================")

    prepared_text = prepare_text(word1)[0]
    posting_list = get_posting_list(prepared_text)
    # posting_list = {3014:{'title':[...] , 'text':[...]}}

    #     print ("posting list for input" , prepared_text, "is :", posting_list , "with length:" , len (posting_list))
    print("number of ocurrences of the word", word1, " in documents = ", get_count(posting_list))
    print('docs with the word:', sorted(list(posting_list.keys())))


test_get_posting_list()


def test_doc_remove():
    print("\n============ testing 'doc_remove' ================================================")

    posting_list = get_posting_list(prepare_text(word1)[0])
    print("length of posting list for word", word1, "before removing doc", doc_id, ":", length_of_posting_list(posting_list))

    delete_document_from_indexes('Persian.xml', doc_id)

    posting_list = get_posting_list(prepare_text(word1)[0])
    print("length of posting list for word", word1, "after removing doc", doc_id, ":", length_of_posting_list(posting_list))

    posting_list = get_posting_list(prepare_text(word2)[0])
    print("length of posting list for word", word2, "before removing doc", doc_id2, ":", length_of_posting_list(posting_list))

    delete_document_from_indexes('Persian.xml', doc_id2)

    posting_list = get_posting_list(prepare_text(word2)[0])
    print("length of posting list for word", word2, "after removing doc", doc_id2, ":", length_of_posting_list(posting_list))


test_doc_remove()


def test_doc_add():
    print("\n============ testing 'doc_add' ================================================")

    posting_list = get_posting_list(prepare_text(word1)[0])
    print("length of posting list for word", word1, "before adding doc", doc_id, ":", length_of_posting_list(posting_list))
    print("number of ocurrences for ", word1, ":", get_count(posting_list))

    add_document_to_indexes('Persian.xml', doc_id)

    posting_list = get_posting_list(prepare_text(word1)[0])
    print("length of posting list for word", word1, "after adding doc", doc_id, ":", length_of_posting_list(posting_list))
    print("number of ocurrences for ", word1, ":", get_count(posting_list))

    posting_list = get_posting_list(prepare_text(word2)[0])
    print("length of posting list for word", word2, "before adding doc", doc_id2, ":", length_of_posting_list(posting_list))

    add_document_to_indexes('Persian.xml', doc_id2)

    posting_list = get_posting_list(prepare_text(word2)[0])
    print("length of posting list for word", word2, "after adding doc", doc_id2, ":", length_of_posting_list(posting_list))


test_doc_add()


def test_save_and_load():
    print("\n============ testing save and load methods ========================================")

    destination = "storage/backup.txt"

    posting_list = get_posting_list(prepare_text(word1)[0])
    print("length of posting list for word", word1, "before saving:", length_of_posting_list(posting_list))
    print("number of ocurrences for ", word1, ":", get_count(posting_list))

    save_index(destination)
    load_index(destination)

    posting_list = get_posting_list(prepare_text(word1)[0])
    print("length of posting list for word", word1, "after loading:", length_of_posting_list(posting_list))
    print("number of ocurrences for ", word1, ":", get_count(posting_list))


test_save_and_load()


def test_search():

    ##################################
    ## Do not change this part
    ##################################
    query = 'سیاره های بزرگ "منظومه شمسی"'
    method = "ltc-lnc"
    ##################################

    relevant_docs = search(query, method)
    print(relevant_docs)

test_search()


def test_detailed_search():
    ##################################
    ## Do not change this part
    ##################################
    title_query = 'فهرست شهرهای ایران'
    text_query = 'استان گیلان شهرستان لنگرود'
    ##################################

    relevant_docs = detailed_search(title_query, text_query)
    print(relevant_docs)


test_detailed_search()

try:
    python_open
    print("Already done!")
except NameError:
    python_open = open
    def open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None):
        encoding="utf-8"
        return python_open(file, mode=mode, buffering=buffering, encoding=encoding, errors=errors, newline=newline, closefd=closefd, opener=opener)


##################################
## Do not change this part
##################################
test_docs = ['all', 1, 2, 3]
functions = {'R_Precision': R_Precision, 'F_measure': F_measure, 'MAP': MAP, 'NDCG': NDCG}
##################################

for doc in test_docs:
    print("{}\ndoc:\t{}".format('-' * 30, doc))
    for f in functions.keys():
        out = functions[f](doc)
        print("{:11}:\t{:.2f}".format(f, out))

##################################
## Do not change this part
##################################
test_docs = [1, 2, 3]
rels = [
    [6753, 7134, 6978, 7136, 4530, 6798, 6885, 5381, 6900, 4537, 5509, 6794, 4094, 6417, 3666, 5967],
    [6753, 5509, 4718, 6798, 6850, 6417, 6978, 6871],
    list(range(20))
]
outputs = [{'R_Precision': 1.0, 'F_measure': 0.967741935483871, 'MAP': 0.9375, 'NDCG': 0.9635640110263509},
           {'R_Precision': 0.4444444444444444, 'F_measure': 0.6153846153846153, 'MAP': 0.4444444444444444, 'NDCG': 0.6313802022799658},
           {'R_Precision': 0.0, 'F_measure': 0.0, 'MAP': 0.0, 'NDCG': 0.0}]

functions = {'R_Precision':R_Precision, 'F_measure':F_measure, 'MAP':MAP, 'NDCG': NDCG}
##################################
idx = 0

ds = detailed_search
s = search
def detailed_search(title_query, text_query, method="ltn-lnn"):
    return rels[idx]

def search(query, method="ltn-lnn", weight=2):
    return rels[idx]

for f in functions.keys():
    print("{}\n{}:".format('-'*30, f))
    idx = 0
    for doc in test_docs:
        out = functions[f](str(doc))
        expected = outputs[idx][f]
        print("{}:\t{:.2f}\t{}".format(doc, out, abs(out-expected)<1e-3))
        idx += 1

detailed_search = ds
search = s